# Limpieza de Datos

#### 1. Corregir Tipos de Datos
- Asegúrate de que cada columna tenga el tipo de dato correcto (fechas, números, categorías).

#### 2. Manejo de Valores Nulos
- Elimina o imputa (rellena) los valores nulos según el contexto y la importancia de la columna.

#### 3. Eliminar Duplicados
- Elimina filas duplicadas para evitar sesgos en el análisis.

#### 4. Corregir Inconsistencias
- Unifica formatos de texto (mayúsculas/minúsculas, espacios, tildes).
- Corrige errores ortográficos en categorías.

#### 5. Tratar Outliers
- Analiza los outliers detectados en el EDA y decide si eliminarlos, corregirlos o dejarlos.

#### 6. Crear Nuevas Variables (si aplica)
- Si necesitas variables derivadas (por ejemplo, año/mes de una fecha), créalas en esta etapa.

In [48]:
import pandas as pd  # Data manipulation
import numpy as np  # Numerical operations
import matplotlib.pyplot as plt  # Data visualization
import seaborn as sns  # Data visualization
import plotly.express as px  # Interactive data visualization

import sys
sys.path.append('..')  # Ajusta el path si es necesario

from main import cargar_datos, guardar_datos

In [49]:
# Cargar datos
datos = cargar_datos()

In [50]:
datos.head()

,Order ID,Date,Product,Category,Price,Quantity,Total Sales,Customer Name,Customer Location,Payment Method,Status
0,ORD0001,14-03-25,Running Shoes,Footwear,60,3,180,Emma Clark,New York,Debit Card,Cancelled
1,ORD0002,20-03-25,Headphones,Electronics,100,4,400,Emily Johnson,San Francisco,Debit Card,Pending
2,ORD0003,15-02-25,Running Shoes,Footwear,60,2,120,John Doe,Denver,Amazon Pay,Cancelled
3,ORD0004,19-02-25,Running Shoes,Footwear,60,3,180,Olivia Wilson,Dallas,Credit Card,Pending
4,ORD0005,10-03-25,Smartwatch,Electronics,150,3,450,Emma Clark,New York,Debit Card,Pending


## Desarrollo

#### 1. Corregir Tipos de Datos
- Asegúrate de que cada columna tenga el tipo de dato correcto (fechas, números, categorías).

In [51]:
# Ver tipos de datos
datos.dtypes

Order ID             object
Date                 object
Product              object
Category             object
Price                 int64
Quantity              int64
Total Sales           int64
Customer Name        object
Customer Location    object
Payment Method       object
Status               object
dtype: object

**Interpretación**:

* Debemos cambiar el tipo de dato de *Date* al correcto

In [52]:
# Cambiar tipo de dato de date
datos['Date'] = pd.to_datetime(datos['Date'], format='%d-%m-%y')
datos.head()

,Order ID,Date,Product,Category,Price,Quantity,Total Sales,Customer Name,Customer Location,Payment Method,Status
0,ORD0001,2025-03-14,Running Shoes,Footwear,60,3,180,Emma Clark,New York,Debit Card,Cancelled
1,ORD0002,2025-03-20,Headphones,Electronics,100,4,400,Emily Johnson,San Francisco,Debit Card,Pending
2,ORD0003,2025-02-15,Running Shoes,Footwear,60,2,120,John Doe,Denver,Amazon Pay,Cancelled
3,ORD0004,2025-02-19,Running Shoes,Footwear,60,3,180,Olivia Wilson,Dallas,Credit Card,Pending
4,ORD0005,2025-03-10,Smartwatch,Electronics,150,3,450,Emma Clark,New York,Debit Card,Pending


#### 2. Manejo de Valores Nulos
- Elimina o imputa (rellena) los valores nulos según el contexto y la importancia de la columna.

In [53]:
datos.isnull().sum()

Order ID             0
Date                 0
Product              0
Category             0
Price                0
Quantity             0
Total Sales          0
Customer Name        0
Customer Location    0
Payment Method       0
Status               0
dtype: int64

* *No hay NULOS*

#### 3. Eliminar Duplicados
- Elimina filas duplicadas para evitar sesgos en el análisis.

In [54]:
duplicados = datos.duplicated().sum()
duplicados

if duplicados == 0:
    print("No hay duplicados en el DataFrame.")
else:
    print(f"Hay {duplicados} duplicados en el DataFrame.")
    datos = datos.drop_duplicates()
    print("Duplicados eliminados.")

No hay duplicados en el DataFrame.


#### 4. Corregir Inconsistencias
- Unifica formatos de texto (mayúsculas/minúsculas, espacios, tildes).
- Corrige errores ortográficos en categorías.

In [55]:
# Unificar formatos de texto (mayúsculas/minúsculas, espacios, tildes) para los campos de texto
# Determinamos las columnas de tipo object
columnas_texto = datos.select_dtypes(include=['object']).columns
# Aplicamos la normalización a cada columna de texto
for col in columnas_texto:
    datos[col] = datos[col].str.title()  # Convertir a título (capitalizar la primera letra de cada palabra)
    datos[col] = datos[col].str.strip()  # Eliminar espacios al inicio y al final
    datos[col] = datos[col].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')  # Eliminar tildes
    

In [56]:
datos.head()

,Order ID,Date,Product,Category,Price,Quantity,Total Sales,Customer Name,Customer Location,Payment Method,Status
0,Ord0001,2025-03-14,Running Shoes,Footwear,60,3,180,Emma Clark,New York,Debit Card,Cancelled
1,Ord0002,2025-03-20,Headphones,Electronics,100,4,400,Emily Johnson,San Francisco,Debit Card,Pending
2,Ord0003,2025-02-15,Running Shoes,Footwear,60,2,120,John Doe,Denver,Amazon Pay,Cancelled
3,Ord0004,2025-02-19,Running Shoes,Footwear,60,3,180,Olivia Wilson,Dallas,Credit Card,Pending
4,Ord0005,2025-03-10,Smartwatch,Electronics,150,3,450,Emma Clark,New York,Debit Card,Pending


#### 5. Tratar Outliers
- Analiza los outliers detectados en el EDA y decide si eliminarlos, corregirlos o dejarlos.

In [57]:
# Identificar el número de outliers usando IQR
def identificar_outliers_iqr(df, columna):
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    outliers = df[(df[columna] < limite_inferior) | (df[columna] > limite_superior)]
    return outliers
outliers_cantidad = identificar_outliers_iqr(datos, 'Total Sales')
outliers_cantidad

,Order ID,Date,Product,Category,Price,Quantity,Total Sales,Customer Name,Customer Location,Payment Method,Status
21,Ord0022,2025-03-07,Refrigerator,Home Appliances,1200,4,4800,Olivia Wilson,Houston,Credit Card,Pending
61,Ord0062,2025-02-10,Laptop,Electronics,800,5,4000,Olivia Wilson,San Francisco,Paypal,Completed
63,Ord0064,2025-02-13,Refrigerator,Home Appliances,1200,4,4800,Emily Johnson,Denver,Paypal,Pending
68,Ord0069,2025-02-25,Refrigerator,Home Appliances,1200,4,4800,David Lee,Boston,Gift Card,Pending
73,Ord0074,2025-03-25,Refrigerator,Home Appliances,1200,4,4800,Jane Smith,Dallas,Gift Card,Cancelled
92,Ord0093,2025-03-06,Refrigerator,Home Appliances,1200,5,6000,David Lee,Denver,Paypal,Cancelled
119,Ord0120,2025-02-16,Laptop,Electronics,800,5,4000,Emily Johnson,Denver,Paypal,Completed
125,Ord0126,2025-02-04,Refrigerator,Home Appliances,1200,5,6000,Olivia Wilson,Chicago,Gift Card,Pending
154,Ord0155,2025-02-05,Refrigerator,Home Appliances,1200,4,4800,Sophia Miller,Seattle,Credit Card,Pending
167,Ord0168,2025-03-24,Laptop,Electronics,800,5,4000,Michael Brown,Miami,Debit Card,Pending


In [58]:
# Aplicamos a todas las columnas y monstramos para cada una un array con los outliers
for col in datos.select_dtypes(include=[np.number]).columns:
    outliers = identificar_outliers_iqr(datos, col)
    print(f"Columna: {col}, Número de outliers: {len(outliers)}")
    print(outliers[[col]])
    

Columna: Price, Número de outliers: 0
Empty DataFrame
Columns: [Price]
Index: []
Columna: Quantity, Número de outliers: 0
Empty DataFrame
Columns: [Quantity]
Index: []
Columna: Total Sales, Número de outliers: 11
     Total Sales
21          4800
61          4000
63          4800
68          4800
73          4800
92          6000
119         4000
125         6000
154         4800
167         4000
230         4000


**Conclusión**:

* *No hay outliers aunque según nos muestre que hay 11 outliers en la columna `Total Sales`*
* *En conlusión según el IQR hay outliers pero son falsos outliers*

#### 6. Crear Nuevas Variables (si aplica)
- Si necesitas variables derivadas (por ejemplo, año/mes de una fecha), créalas en esta etapa.

In [59]:
# Crear variables según la columna 'Date' y agruparlas en un tres nuevas columnas
datos['Year'] = datos['Date'].dt.year
datos['Month'] = datos['Date'].dt.month
datos['Day'] = datos['Date'].dt.day
datos.head()

,Order ID,Date,Product,Category,Price,Quantity,Total Sales,Customer Name,Customer Location,Payment Method,Status,Year,Month,Day
0,Ord0001,2025-03-14,Running Shoes,Footwear,60,3,180,Emma Clark,New York,Debit Card,Cancelled,2025,3,14
1,Ord0002,2025-03-20,Headphones,Electronics,100,4,400,Emily Johnson,San Francisco,Debit Card,Pending,2025,3,20
2,Ord0003,2025-02-15,Running Shoes,Footwear,60,2,120,John Doe,Denver,Amazon Pay,Cancelled,2025,2,15
3,Ord0004,2025-02-19,Running Shoes,Footwear,60,3,180,Olivia Wilson,Dallas,Credit Card,Pending,2025,2,19
4,Ord0005,2025-03-10,Smartwatch,Electronics,150,3,450,Emma Clark,New York,Debit Card,Pending,2025,3,10


In [60]:
# Guardar datos limpios
guardar_datos(datos)